Necessary imports

In [ ]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve


Create SparkSession if not already in pyspark

In [ ]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[2]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [ ]:
import time


embeddingsFile = 'PubMed-shuffle-win-2.bin'
embeddingsUrl = 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/PubMed-shuffle-win-2.bin'
# this may take a couple minutes
if not Path(embeddingsFile).is_file():
    urlretrieve(embeddingsUrl, embeddingsFile)

documentAssembler = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")\
    
tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")
    
pretrained_pos = PerceptronModel().pretrained()\
    .setInputCols(["document", "token"])\
    .setOutputCol("pos")
    
pretrained_ner = NerCrfModel().pretrained()\
    .setInputCols(["document", "token", "pos"])\
    .setOutputCol("ner")

assertion = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols(["document"])\
    .setOutputCol("assertion")\
    .setBefore(11)\
    .setAfter(13)\
    .setEmbeddingsSource(embeddingsFile,200,3)\
    .setNerCol("ner")\
    .setTargetNerLabels(["I-PER"])\
    .setExhaustiveNerMode(False)

finisher = Finisher() \
    .setInputCols(["ner", "assertion"]) \
    .setIncludeKeys(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    pretrained_pos,
    pretrained_ner,
    assertion,
    finisher
  ])


This example doesn't really make sense, since we are using a standard NER model, so we create a dataset with Person just to make sure assertion status uses such TargetNerLabel

In [ ]:
#Load the input data to be annotated
data = spark.createDataFrame([
    ["hello Peter how are you? And how is Robert Douglas?", "Affirmed"],
    ["master of yards", "Negated"],
    ["what is this", "Negated"],
    ["Michael Jordan is a good person", "Negated"],
    ["Robert Deniro is not a good person", "Negated"],
    ["Learn from Carlos Rodriguez", "Negated"],
    ["Who is Jhon here?", "Negated"],
    ["my friend Lucas", "Affirmed"]
    ]).toDF("sentence", "label")
data.show()

We fit the data

In [ ]:
start = time.time()
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")
print (time.time() - start)

Prediction with finisher output

In [ ]:
result = model.transform(data)
start = time.time()
result.select("sentence", "finished_ner", "finished_assertion").show()
print (time.time() - start)